<a href="https://colab.research.google.com/github/MauricioFBL/capstone_project/blob/main/notebooks/transformations_casptone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
#!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

import findspark
findspark.init()
findspark.find()

from pyspark.sql import DataFrame, SparkSession 



spark = SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()

spark

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [12]:
movie_review_df = (spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv('movie_review.csv'))

In [3]:
movie_review_df.show(5)

+-----+--------------------+---------+
|  cid|          review_str|id_review|
+-----+--------------------+---------+
|13756|Once again Mr. Co...|        1|
|15738|This is an exampl...|        2|
|15727|First of all I ha...|        3|
|17954|Not even the Beat...|        4|
|16579|Brass pictures (m...|        5|
+-----+--------------------+---------+
only showing top 5 rows



In [13]:
tokenizer = Tokenizer(inputCol='review_str', outputCol='Words')
# remover = StopWordsRemover(inputCol='Words', outputCol='words_filtered')
movie_review_df = tokenizer.transform(movie_review_df)
# movie_review_df = remover.transform(movie_review_df)
movie_review_df.show(5)

+-----+--------------------+---------+--------------------+
|  cid|          review_str|id_review|               Words|
+-----+--------------------+---------+--------------------+
|13756|Once again Mr. Co...|        1|[once, again, mr....|
|15738|This is an exampl...|        2|[this, is, an, ex...|
|15727|First of all I ha...|        3|[first, of, all, ...|
|17954|Not even the Beat...|        4|[not, even, the, ...|
|16579|Brass pictures (m...|        5|[brass, pictures,...|
+-----+--------------------+---------+--------------------+
only showing top 5 rows



In [14]:

remover = StopWordsRemover(inputCol='Words', outputCol='words_filtered')
movie_review_df = remover.transform(movie_review_df)
movie_review_df.show(5)

+-----+--------------------+---------+--------------------+--------------------+
|  cid|          review_str|id_review|               Words|      words_filtered|
+-----+--------------------+---------+--------------------+--------------------+
|13756|Once again Mr. Co...|        1|[once, again, mr....|[mr., costner, dr...|
|15738|This is an exampl...|        2|[this, is, an, ex...|[example, majorit...|
|15727|First of all I ha...|        3|[first, of, all, ...|[first, hate, mor...|
|17954|Not even the Beat...|        4|[not, even, the, ...|[even, beatles, w...|
|16579|Brass pictures (m...|        5|[brass, pictures,...|[brass, pictures,...|
+-----+--------------------+---------+--------------------+--------------------+
only showing top 5 rows



In [15]:
movie_review_df.printSchema()

root
 |-- cid: integer (nullable = true)
 |-- review_str: string (nullable = true)
 |-- id_review: integer (nullable = true)
 |-- Words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- words_filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [20]:
from pyspark.sql.functions import *

In [21]:
movie_review_df = movie_review_df.withColumn("positive_review",when(array_contains(col("words_filtered"),"good"),1).otherwise(0))
movie_review_df.show(5)

+-----+--------------------+---------+--------------------+--------------------+-----------+
|  cid|          review_str|id_review|               Words|      words_filtered|review_type|
+-----+--------------------+---------+--------------------+--------------------+-----------+
|13756|Once again Mr. Co...|        1|[once, again, mr....|[mr., costner, dr...|          0|
|15738|This is an exampl...|        2|[this, is, an, ex...|[example, majorit...|          0|
|15727|First of all I ha...|        3|[first, of, all, ...|[first, hate, mor...|          0|
|17954|Not even the Beat...|        4|[not, even, the, ...|[even, beatles, w...|          0|
|16579|Brass pictures (m...|        5|[brass, pictures,...|[brass, pictures,...|          1|
+-----+--------------------+---------+--------------------+--------------------+-----------+
only showing top 5 rows



In [26]:
# dt.datetime.now()
movie_review_df = movie_review_df.withColumn('insert_date ', current_timestamp())
movie_review_df.show(5)

+-----+--------------------+---------+--------------------+--------------------+-----------+--------------------+
|  cid|          review_str|id_review|               Words|      words_filtered|review_type|        insert_date |
+-----+--------------------+---------+--------------------+--------------------+-----------+--------------------+
|13756|Once again Mr. Co...|        1|[once, again, mr....|[mr., costner, dr...|          0|2022-07-23 17:49:...|
|15738|This is an exampl...|        2|[this, is, an, ex...|[example, majorit...|          0|2022-07-23 17:49:...|
|15727|First of all I ha...|        3|[first, of, all, ...|[first, hate, mor...|          0|2022-07-23 17:49:...|
|17954|Not even the Beat...|        4|[not, even, the, ...|[even, beatles, w...|          0|2022-07-23 17:49:...|
|16579|Brass pictures (m...|        5|[brass, pictures,...|[brass, pictures,...|          1|2022-07-23 17:49:...|
+-----+--------------------+---------+--------------------+--------------------+--------

In [27]:
moviereviews = movie_review_df.drop('review_str','Words','words_filtered')

In [28]:
moviereviews.show(5)

+-----+---------+-----------+--------------------+
|  cid|id_review|review_type|        insert_date |
+-----+---------+-----------+--------------------+
|13756|        1|          0|2022-07-23 17:50:...|
|15738|        2|          0|2022-07-23 17:50:...|
|15727|        3|          0|2022-07-23 17:50:...|
|17954|        4|          0|2022-07-23 17:50:...|
|16579|        5|          1|2022-07-23 17:50:...|
+-----+---------+-----------+--------------------+
only showing top 5 rows



# Log transformations

In [29]:
sourceDf = spark.read.csv('log_reviews.csv', sep=',', header=True, inferSchema=True, multiLine=True)
sourceDf.show(5)

+---------+--------------------+
|id_review|                 log|
+---------+--------------------+
|        1|<reviewlog><log><...|
|        2|<reviewlog><log><...|
|        3|<reviewlog><log><...|
|        4|<reviewlog><log><...|
|        5|<reviewlog><log><...|
+---------+--------------------+
only showing top 5 rows



In [ ]:
#   import com.databricks.spark.xml._
#   import com.databricks.spark.xml.functions.from_xml

#   val spark = SparkSession.builder().master("local[*]").getOrCreate()
#   import spark.implicits._
#   spark.sparkContext.setLogLevel("ERROR")

#   val df = // Read csv file
  
#   // Assuming your xml content column name is xmldata
#   val xmlSchema = schema_of_xml(df.select("xmldata").as[String])

#   df.withColumn("xmldata", from_xml('xmldata, xmlSchema))
#     .select("*", "xmldata.ab")
#     .selectExpr(df.columns.diff(Array("xmldata")) ++
#       Array("ab[0]._a as name", "ab[0]._b as id", "ab[1]._a as manager_name", "ab[1]._b as manager_id"): _*)
#     .show(false)

In [32]:
!$SPARK_HOME/bin/spark-shell --packages com.databricks:spark-xml_2.12:0.15.0

:: loading settings :: url = jar:file:/content/spark-3.2.2-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c1fc4ee6-2798-4c49-83ff-ad94decf51a8;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.15.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
downloading https://repo1.maven.org/maven2/com/databricks/spark-xml_2.12/0.15.0/spark-xml_2.12-0.15.0.jar ...
	[SUCCESSFUL ] com.databricks#spark-xml_2.12;0.15.0!spark-xml_2.12.jar (44ms)
downloading https://repo1.maven.org/maven2/commons-io/commons-io/2.11.0/commons-io-2.11.0.jar ...
	[SUCCESSFUL ] commons-io#commons-io;2.11.0!commons-io.jar (38ms)
downloading https://re

In [37]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.12.0 pyspark-shell'

In [38]:
# import databricks.spark.xml
# import databricks.spark.xml.functions.from_xml
# from pyspark

+---------+--------------------+
|id_review|                 log|
+---------+--------------------+
|        1|<reviewlog><log><...|
|        2|<reviewlog><log><...|
|        3|<reviewlog><log><...|
|        4|<reviewlog><log><...|
|        5|<reviewlog><log><...|
|        6|<reviewlog><log><...|
|        7|<reviewlog><log><...|
|        8|<reviewlog><log><...|
|        9|<reviewlog><log><...|
|       10|<reviewlog><log><...|
+---------+--------------------+
only showing top 10 rows



In [41]:

from pyspark.sql.types import *

In [46]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *
from decimal import Decimal
appName = "Python Example - PySpark Read XML"
master = "local"

# Create Spark session
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.11:0.9.0") \
    .getOrCreate()

In [47]:
log_review_df = (spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv('log_reviews.csv'))
log_review_df.show(10)

+---------+--------------------+
|id_review|                 log|
+---------+--------------------+
|        1|<reviewlog><log><...|
|        2|<reviewlog><log><...|
|        3|<reviewlog><log><...|
|        4|<reviewlog><log><...|
|        5|<reviewlog><log><...|
|        6|<reviewlog><log><...|
|        7|<reviewlog><log><...|
|        8|<reviewlog><log><...|
|        9|<reviewlog><log><...|
|       10|<reviewlog><log><...|
+---------+--------------------+
only showing top 10 rows



In [ ]:
xmlSchema = sche (log_review_df.select("xmldata").as[String])

In [56]:
# xmlSchema = spark.read.format('xml').options(rowTag='log').load(log_review_df.select("log"))
df = spark.read.format("xml").load(log_review_df.select("log"))

AttributeError: ignored

In [ ]:
log_review_df = (spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv('log_reviews.csv'))
log_review_df.show(10)

+---------+--------------------+
|id_review|                 log|
+---------+--------------------+
|        1|<reviewlog><log><...|
|        2|<reviewlog><log><...|
|        3|<reviewlog><log><...|
|        4|<reviewlog><log><...|
|        5|<reviewlog><log><...|
|        6|<reviewlog><log><...|
|        7|<reviewlog><log><...|
|        8|<reviewlog><log><...|
|        9|<reviewlog><log><...|
|       10|<reviewlog><log><...|
+---------+--------------------+
only showing top 10 rows



In [ ]:
schema

In [ ]:


#   df.withColumn("xmldata", from_xml('xmldata, xmlSchema))
#     .select("*", "xmldata.ab")
#     .selectExpr(df.columns.diff(Array("xmldata")) ++
#       Array("ab[0]._a as name", "ab[0]._b as id", "ab[1]._a as manager_name", "ab[1]._b as manager_id"): _*)
#     .show(false)

In [ ]:
log_review_df = (spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv('log_reviews.csv'))

In [5]:
user_purchase_df = (spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv('user_purchase.csv'))

In [6]:
user_purchase_df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows

